In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from lif import LIFLayer

dt: float = 1e-3
runtime: float = 0.1
N: int = 5
T: int = int(runtime / dt)
print(f'Simulating {runtime} seconds of activity over {T} cycles.')

# create a layer of N neurons
#weights = torch.rand(size=(1, N))
weights = torch.linspace(0, 1, steps=N)
lif_layer = LIFLayer(weights)

# create a step input of size T
step_input = torch.ones((T, 1), dtype=torch.float)

# Preallocate tensor storage for outputs and inhibitory_states
outputs = torch.zeros(T, N)
inhibitory_states = torch.zeros(T, 1)

# simulate the layer and store the outputs
time = torch.linspace(0, T * lif_layer.dt, T)
for i, t in enumerate(time):
    spikes = lif_layer(step_input[i])
    outputs[i] = lif_layer.state
    inhibitory_states[i] = lif_layer.inhibitory_state

# Convert tensors to numpy for plotting
outputs_np = outputs.numpy()
inhibitory_states_np = inhibitory_states.numpy()

fig, ax1 = plt.subplots(figsize=(10, 4))

color = 'tab:red'
ax1.set_xlabel('Time')
ax1.set_ylabel('$V_m$', color=color)
for i in range(N):
    ax1.plot(time, outputs_np[:, i], label=f'$\omega$={weights[i]}', color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.legend(loc='upper left')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Current/Inhibition', color='tab:blue')  # we already handled the x-label with ax1
#ax2.plot(time, step_input.numpy(), color='tab:green', label='Input Current')
ax2.plot(time, inhibitory_states_np, color='tab:blue', label='Inhibition')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.legend(loc='upper right')

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

Simulating 0.1 seconds of activity over 100 cycles.
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\liamb\anaconda3\envs\thesis_env\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\liamb\AppData\Local\Temp\ipykernel_10684\1129830530.py", line 15, in <module>
    lif_layer = LIFLayer(weights)
  File "c:\Users\liamb\code\FFSNN\lif.py", line 13, in __init__
NameError: name 'torch' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\liamb\anaconda3\envs\thesis_env\lib\site-packages\IPython\core\interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\liamb\anaconda3\envs\thesis_env\lib\site-packages\IPython\core\ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\liamb\anaconda3\envs\thesis_env\lib\site-packages\IPython\core\ultratb.py", li

In [3]:
dt: float = 1e-4
runtime: float = 0.1
N: int = 5
T: int = int(runtime / dt)
print(f'Simulating {runtime} seconds of activity over {T} cycles.')

# create a layer of N neurons
#weights = torch.rand(size=(1, N))
weights = torch.linspace(0, 1, steps=N)
lif_layer = LIFLayer(weights)

# create a step input of size T
step_input = torch.ones((T, 1), dtype=torch.float)

# Preallocate tensor storage for spike counts
spikes_count = torch.zeros((N,), dtype=torch.int)

# simulate the layer and store the outputs
time = torch.linspace(0, T * lif_layer.dt, T)
for i, t in enumerate(time):
    spikes = lif_layer(step_input[i])
    spikes_count += spikes.int()

# Convert tensors to numpy for plotting
spikes_count_np = spikes_count.numpy()

Simulating 0.1 seconds of activity over 1000 cycles.
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\liamb\anaconda3\envs\thesis_env\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\liamb\AppData\Local\Temp\ipykernel_10684\1881424571.py", line 10, in <module>
    lif_layer = LIFLayer(weights)
  File "c:\Users\liamb\code\FFSNN\lif.py", line 13, in __init__
NameError: name 'torch' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\liamb\anaconda3\envs\thesis_env\lib\site-packages\IPython\core\interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\liamb\anaconda3\envs\thesis_env\lib\site-packages\IPython\core\ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\liamb\anaconda3\envs\thesis_env\lib\site-packages\IPython\core\ultratb.py", li